In [ ]:
# To Run jupyter notebook, you need to download xview training dataset and training labels from  https://challenge.xviewdataset.org/download-links
# after creating

In [4]:
import geojson
from datasets import Image as Im
import pyarrow as pa
import pandas as pd
from datasets import Dataset
import datasets
from PIL import Image
import numpy as np

In [13]:
with open("xview/train_labels/xView_train.geojson") as f:
    gj = geojson.load(f)
df = pd.json_normalize([feat for feat in gj["features"]])
df = df.reset_index()
del gj

In [14]:
df = df[df["properties.image_id"] != "1395.tif"]
df = df.reset_index()
df.drop(["level_0", "index"], axis=1, inplace=True)

In [15]:
df["properties.image_location"] = "xview/train_images/" + df["properties.image_id"]
df["list_of_cooridates"] = df["properties.bounds_imcoords"].str.split(",")
df["list_of_cooridates"] = df["list_of_cooridates"].apply(
    lambda x: [int(el) for el in x]
)
# df['shape'] = df['properties.image_location'].progress_apply(lambda x: np.array(Image.open(x)).shape)
df["first"] = df["list_of_cooridates"].map(lambda x: int(x[0]))
df["second"] = df["list_of_cooridates"].map(lambda x: int(x[1]))
df["third"] = df["list_of_cooridates"].map(lambda x: int(x[2]))
df["fourth"] = df["list_of_cooridates"].map(lambda x: int(x[3]))
df["area"] = df.apply(
    lambda x: (x["third"] - x["first"]) * (x["fourth"] - x["second"]), axis=1
)

In [16]:
df["shape"] = 0
df_final = pd.DataFrame()
LIST = []
i = 0
for group_name, df2 in df.groupby("properties.image_location"):
    shape_val = list(np.array(Image.open(group_name)).shape)

    df2["shape"] = [shape_val for x in range(0, df2.shape[0])]

    df2["properties.image_location"] = group_name
    LIST.append(df2)
    if len(LIST) > 20:
        df_concat = pd.concat(LIST)
        df_final = pd.concat([df_final, df_concat])
        LIST = []
        print("finished with " + str(i))
    i += 1
if len(LIST) > 0:
    df_concat = pd.concat(LIST)
    df_final = pd.concat([df_final, df_concat])

df = df_final.reset_index()
del df_final, df2, df_concat

finished with 20
finished with 41
finished with 62
finished with 83
finished with 104
finished with 125
finished with 146
finished with 167
finished with 188
finished with 209
finished with 230
finished with 251
finished with 272
finished with 293
finished with 314
finished with 335
finished with 356
finished with 377
finished with 398
finished with 419
finished with 440
finished with 461
finished with 482
finished with 503
finished with 524
finished with 545
finished with 566
finished with 587
finished with 608
finished with 629
finished with 650
finished with 671
finished with 692
finished with 713
finished with 734
finished with 755
finished with 776
finished with 797
finished with 818
finished with 839


In [17]:
df["shape_1"] = df["shape"].map(lambda x: int(x[0]))
df["shape_2"] = df["shape"].map(lambda x: int(x[1]))

In [18]:
df["third"] = df[["third", "shape_2"]].min(axis=1)
df["fourth"] = df[["fourth", "shape_1"]].min(axis=1)
df["first"] = df["first"].clip(0, None)
df["second"] = df["second"].clip(0, None)

In [19]:
df = df.drop(df[((df["first"] == df["third"]) | (df["second"] == df["fourth"]))].index)
df = df.reset_index()

In [20]:
print("check to make sure crop doesn't take place outside image")
print(
    df[((df.third > df["shape_2"]) | (df.fourth > df["shape_1"]))][
        ["list_of_cooridates", "shape"]
    ]
)
print(
    df[((df["first"] > df["third"]) | (df["second"] > df["fourth"]))][
        ["list_of_cooridates", "shape"]
    ]
)
print(df[((df["first"] < 0) | (df["second"] < 0))][["list_of_cooridates", "shape"]])
print(
    df[((df["first"] == df["third"]) | (df["second"] == df["fourth"]))][
        ["list_of_cooridates", "shape"]
    ]
)

check to make sure crop doesn't take place outside image
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []


In [21]:
df

,level_0,index,type,geometry.type,geometry.coordinates,properties.bounds_imcoords,properties.edited_by,properties.cat_id,properties.type_id,properties.ingest_time,...,properties.image_location,list_of_cooridates,first,second,third,fourth,area,shape,shape_1,shape_2
0,0,144655,Feature,Polygon,"[[[-23.475366, 14.95406], [-23.475366, 14.9540...","1409,732,1430,743",wwmabelcontreras,10400100290EB700,73,2017/07/28 16:01:30.348+00,...,xview/train_images/10.tif,"[1409, 732, 1430, 743]",1409,732,1430,743,231,"[3228, 3320, 3]",3228,3320
1,1,144656,Feature,Polygon,"[[[-23.475992, 14.954804], [-23.475992, 14.954...","1188,466,1216,480",wwramdiebermudez,10400100290EB700,23,2017/07/28 17:07:38.880+00,...,xview/train_images/10.tif,"[1188, 466, 1216, 480]",1188,466,1216,480,392,"[3228, 3320, 3]",3228,3320
2,2,144657,Feature,Polygon,"[[[-23.474715, 14.955827], [-23.474715, 14.956...","1639,37,1706,119",wwkimberlypinero,10400100290EB700,73,2017/08/09 12:11:57.210+00,...,xview/train_images/10.tif,"[1639, 37, 1706, 119]",1639,37,1706,119,5494,"[3228, 3320, 3]",3228,3320
3,3,144658,Feature,Polygon,"[[[-23.474944, 14.954986], [-23.474944, 14.954...","1558,416,1620,474",wwnicolecenteno,10400100290EB700,73,2017/08/10 18:26:49.056+00,...,xview/train_images/10.tif,"[1558, 416, 1620, 474]",1558,416,1620,474,3596,"[3228, 3320, 3]",3228,3320
4,4,144659,Feature,Polygon,"[[[-23.474843, 14.95452], [-23.474843, 14.9544...","1594,581,1622,600",wwnicolecenteno,10400100290EB700,73,2017/08/10 18:28:49.332+00,...,xview/train_images/10.tif,"[1594, 581, 1622, 600]",1594,581,1622,600,532,"[3228, 3320, 3]",3228,3320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601769,601801,264570,Feature,Polygon,"[[[10.244514, 36.871548], [10.244514, 36.87158...","1106,150,1125,161",wwariadnagonzalez,104001002B7A7300,63,2017/08/31 20:45:27.506+00,...,xview/train_images/99.tif,"[1106, 150, 1125, 161]",1106,150,1125,161,209,"[2714, 3378, 3]",2714,3378
601770,601802,264571,Feature,Polygon,"[[[10.244582, 36.871562], [10.244582, 36.87160...","1127,144,1152,157",wwariadnagonzalez,104001002B7A7300,63,2017/08/31 20:45:27.506+00,...,xview/train_images/99.tif,"[1127, 144, 1152, 157]",1127,144,1152,157,325,"[2714, 3378, 3]",2714,3378
601771,601803,137871,Feature,Polygon,"[[[-68.89912, -22.157275], [-68.89912, -22.157...","142,1300,203,1339",wwchristopherviera,10400100282F1500,93,2017/08/08 13:42:26.359+00,...,xview/train_images/996.tif,"[142, 1300, 203, 1339]",142,1300,203,1339,2379,"[2769, 2973, 3]",2769,2973
601772,601804,137872,Feature,Polygon,"[[[-68.897841, -22.160358], [-68.897841, -22.1...","534,2245,590,2283",wwchristopherviera,10400100282F1500,93,2017/08/08 13:42:26.359+00,...,xview/train_images/996.tif,"[534, 2245, 590, 2283]",534,2245,590,2283,2128,"[2769, 2973, 3]",2769,2973


In [22]:
df["shape"] = 0
df_final = pd.DataFrame()
LIST = []
i = 0
for group_name, df2 in df.groupby("properties.image_location"):
    df2 = df2.drop(["level_0"], axis=1)
    df2 = df2.reset_index()
    df_append = pd.DataFrame(
        index=range(1), columns=["image_id", "image", "width", "height", "objects"]
    )

    height, width, _ = np.array(Image.open(group_name)).shape
    df_append.at[0, "image_id"] = df2["properties.image_id"][0]
    df_append.at[0, "image"] = group_name
    df_append.at[0, "width"] = width
    df_append.at[0, "height"] = height
    df_append.at[0, "objects"] = dict(
        {
            "id": df2["index"].tolist(),
            "area": df2["area"].tolist(),
            "bbox": df2["list_of_cooridates"].tolist(),
            "category": df2["properties.type_id"].tolist(),
        }
    )

    LIST.append(df_append)
    if len(LIST) > 20:
        df_concat = pd.concat(LIST)
        df_final = pd.concat([df_final, df_concat])
        LIST = []
        print("finished with " + str(i))
    i += 1
if len(LIST) > 0:
    df_concat = pd.concat(LIST)
    df_final = pd.concat([df_final, df_concat])

df = df_final.reset_index()
del df_final, df2, df_concat, df_append

finished with 20
finished with 41
finished with 62
finished with 83
finished with 104
finished with 125
finished with 146
finished with 167
finished with 188
finished with 209
finished with 230
finished with 251
finished with 272
finished with 293
finished with 314
finished with 335
finished with 356
finished with 377
finished with 398
finished with 419
finished with 440
finished with 461
finished with 482
finished with 503
finished with 524
finished with 545
finished with 566
finished with 587
finished with 608
finished with 629
finished with 650
finished with 671
finished with 692
finished with 713
finished with 734
finished with 755
finished with 776
finished with 797
finished with 818
finished with 839


In [23]:
hg_dataset = Dataset(pa.Table.from_pandas(df))
NewDataset = datasets.DatasetDict({"train": hg_dataset})  # ,"val":test_dataset})
newdata = NewDataset["train"].cast_column("image", Im())
NewDataset = datasets.DatasetDict({"train": newdata})  # ,"val":test_dataset['train']})
del newdata, hg_dataset

In [24]:
NewDataset["train"].data.nbytes / 1e9

0.036179418

In [25]:
NewDataset

DatasetDict({
    train: Dataset({
        features: ['index', 'image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 846
    })
})

In [29]:
# NewDataset.push_to_hub('Honaker/xview_dataset',  num_shards={'train': 50, 'test': 0})

Pushing dataset shards to the dataset hub: 100%|██████████| 50/50 [4:20:09<00:00, 312.18s/it]
